### Modules

In [6]:
import requests
from bs4 import BeautifulSoup
import re
import time
from datetime import datetime
import pandas as pd

### Predefine parameters

In [2]:
url = "http://www.moha.gov.my/index.php/ms/kdn1/dir-kdn"

### Start

In [8]:
#main content
start = datetime.now()
content = requests.get(url)
bsContent = BeautifulSoup(content.text, 'html.parser')

#section and href list
sectionList = [re.sub("\n|\t", "", i.a.get_text()) for i in bsContent.findAll("h3", {'class' : "page-header item-title"})]
hrefList = [i.a['href'] for i in bsContent.findAll("h3", {'class' : "page-header item-title"})]

pandaList = []
for no, link in enumerate(hrefList):
    time.sleep(3)
    tempDict = {}
    urlEx = "http://www.moha.gov.my" + link
    content2 = requests.get(urlEx, verify=False)
    bsContent2 = BeautifulSoup(content2.text, 'html.parser')
    
    allA = [i.a for i in bsContent2.find_all("li") if i.a]
    nameList = [re.sub("\n|\t", "", i.get_text()).strip() for i in allA if re.search(re.escape(link + '/'), i['href'])]
    phoneList = [re.sub("\n|\t", "", i.get_text()) for i in bsContent2.findAll("div", {"class": "span3 col-md-3"})]
    
    tempDict['Section'] = sectionList[no]
    tempDict['Name'] = nameList
    tempDict['Phone'] = phoneList
    df = pd.DataFrame(tempDict)
    pandaList.append(df)

#concat df and save csv
dfResult = pd.concat(pandaList).reset_index()
del dfResult['index']

csvFile = "2021-02-25_KDN.csv"
dfResult.to_csv(csvFile, index = False)
time = datetime.now() - start
print("Estimated time: %s"%time)

Estimated time: 0:02:04.862529


In [10]:
dfResult['Section'].unique()

array(['Pejabat Menteri ', 'Pejabat Timbalan Menteri',
       'Pejabat Timbalan Menteri II', 'Pejabat Ketua Setiausaha',
       'Pejabat Timbalan Ketua Setiausaha (Keselamatan)',
       'Pejabat Timbalan Ketua Setiausaha (Pengurusan)',
       'Pejabat Timbalan Ketua Setiausaha (Dasar dan Kawalan)',
       'Agensi Kawalan Sempadan Malaysia (AKSEM)', 'Bahagian Keselamatan',
       'Bahagian Penguatkuasaan dan Kawalan',
       'Bahagian Pengurusan Pekerja Asing',
       'Bahagian Pengurusan Teknologi Maklumat',
       'Bahagian Pengurusan Sumber Manusia', 'Bahagian Antarabangsa',
       'Bahagian Kepenjaraan, Antidadah dan RELA',
       'Pejabat Penasihat Undang-Undang', 'Bahagian Audit Dalam',
       'Bahagian Khidmat Pengurusan dan Aset',
       'Bahagian Hal Ehwal Imigresen',
       'Pejabat Strategik Nasional (NSO) MAPO',
       'Unit Komunikasi Korporat', 'Bahagian Parlimen dan Kabinet',
       'Bahagian Perancangan Strategik', 'Bahagian Pembangunan',
       'Bahagian Kewangan', 'Pej